# Assignment 0: Conical tank emptying time

---

Let's calculate how long it takes to [empty a conical tank](https://www.youtube.com/watch?v=2jQ1jA8uJuU)

The simplification of the continuity equation leads, for a generic $h$, to

\begin{equation}
u_1 D_h^2 = u_1 \left(d + 2h \tan \theta \right)^2 = u_2 d^2 \tag{1} \label{eq_cont}
\end{equation}

\begin{equation}
\Rightarrow \; u_2 = u_1\left(1 + 2\frac{h}{d} \tan \theta \right)^2
\end{equation}

On the other hand, Bernoulli's equation, considering quasi-steady flow and without viscosity,

\begin{equation}
u_1^2 + 2gh = u_2^2 = u_1^2 \left(1 + 2\frac{h}{d} \tan \theta \right)^4
\end{equation}

\begin{equation}
\Rightarrow \; u_1^2 \left[ \left(1 + 2\frac{h}{d} \tan \theta \right)^4 -1 \right] = 2gh
\end{equation}

\begin{equation}
\Rightarrow \; u_1 = \left[ \frac{2gh}{\left(1 + 2\frac{h}{d} \tan \theta \right)^4 -1} \right]^\frac{1}{2} = -\frac{\textrm{d} h}{\textrm{d} t}
\end{equation}

\begin{equation}
\Rightarrow \; \boxed{dt = - \left[ \frac{\left(1 + 2\frac{h}{d} \tan \theta \right)^4 -1}{2gh}\right]^\frac{1} {2} \textrm{d} h} \tag{2} \label{eq:int}
\end{equation}

This can be written dimensionless with a time characteristic $\tau = \sqrt{\frac{d}{2g}}$, a change to dimensionless variable $\xi = \frac{h}{d}$, and a parameter $A = 2\tan \theta$, becoming

\begin{equation}
\frac{\textrm{d} t}{\tau} = - \left[ \frac{\left(1 + A \xi \right)^4 -1}{\xi}\right]^\frac{1 }{2} \textrm{d} \xi
\end{equation}

Use sympy to analytically compute this time, making an estimation with  $d \ll h$ so that it can be supposed that  $\xi \gg 1$.

## Simplified form

A first simplification is to assume that $d \ll h$ at all times, so that $\xi$ is very large. In this case, $(1+A\xi)^4-1 \approx (1+A\xi)^4$ can be considered. This is what would be obtained from directly applying Torricelli's equation,

\begin{equation}
u_2 = \sqrt{2gh}
\end{equation}

In [1]:
import sympy as sp
import numpy as np

In [2]:
t,tau,xi,A,xi_0,xi_f = sp.symbols('t,tau,xi,A,xi_0,xi_f',positive=True, real=True)

In [3]:
f1 = (1+A*xi)**2/sp.sqrt(xi)

In [4]:
f1

(A*xi + 1)**2/sqrt(xi)

In [5]:
f1I = sp.integrate(f1,(xi,xi_0,xi_f))

In [6]:
f1I = f1I.simplify()

In [7]:
f1I

-2*A**2*xi_0**(5/2)/5 + 2*A**2*xi_f**(5/2)/5 - 4*A*xi_0**(3/2)/3 + 4*A*xi_f**(3/2)/3 - 2*sqrt(xi_0) + 2*sqrt(xi_f)

In [8]:
t1 = -tau*f1I

In [9]:
t1

-tau*(-2*A**2*xi_0**(5/2)/5 + 2*A**2*xi_f**(5/2)/5 - 4*A*xi_0**(3/2)/3 + 4*A*xi_f**(3/2)/3 - 2*sqrt(xi_0) + 2*sqrt(xi_f))

Let's assign some values to the variables and get an example numerical result

In [10]:
t1f = sp.lambdify((tau,A,xi_0,xi_f),t1)

In [11]:
d = 0.01 # d = 10 mm
H0 = 0.2 # H0 = 20 cm
Hf = 0
theta = np.deg2rad(20) # theta = 20 grados
g = 9.81 # la gravedad
tauValue = np.sqrt(d/(2*g))
AValue = 2*np.tan(theta)
xi_0Value = H0/d
xi_fValue = Hf/d

In [12]:
t1Value = sp.Float(t1f(tauValue,AValue,xi_0Value,xi_fValue),5)
t1Value

10.722

## More accurate way

Let us assume, however, that we do not make the above approximation. 

In [13]:
f2 = sp.sqrt(((1+A*xi)**4-1)/xi)

In [14]:
f2

sqrt((A*xi + 1)**4 - 1)/sqrt(xi)

This expression is not so easy to integrate

In [15]:
f2I = sp.integrate(f2,(xi,xi_0,xi_f))

In [16]:
f2I

sqrt(A)*Integral(sqrt(A*xi + 2)*sqrt(A**2*xi**2 + 2*A*xi + 2), (xi, xi_0, xi_f))

In fact, it does not have a simple analytic integral, and it must be done numerically. We use mpmath, a library that allows you to do arbitrary precision operations. See http://mpmath.org

In [17]:
import mpmath
f2_mpmath = sp.lambdify((A,xi),f2)

In [18]:
H0 = 0.2 # H0 = 20 cm
Hf = 0
xi_0 = H0/d
xi_f = Hf/d
f2I = mpmath.quad(lambda xi:f2_mpmath(AValue,xi),(xi_0,xi_f))

In [19]:
t2Value = -tauValue*f2I

In [20]:
t2Value = sp.Float(t2Value,5)
t2Value

10.696

As you can see, it takes a little less time to empty. But the difference is negligible.

In [21]:
error = np.abs((t1Value-t2Value)/t1Value)
error

0.0024492

Barely 0.25%